In [ ]:
!pip install stockstats

In [22]:
import ccxt
import os 
import re
import time
import pandas as pd

from stockstats import StockDataFrame as Sdf

# configure exchange
# exchange = getattr(ccxt, 'binance')({
#   'apiKey': os.environ['APIKEY'],
#   'secret': os.environ['SECRET'],
#   'timeout': 10000,
#   'enableRateLimit': True
# })

# load markets and all coin_pairs
# exchange.load_markets()
binance = ccxt.binance()
exchange = binance.load_markets()

In [32]:
def get_historical_data(coin_pair, timeframe):
    """Get Historical data (ohlcv) from a coin_pair
    """
    # optional: exchange.fetch_ohlcv(coin_pair, '1h', since)
    data = binance.fetch_ohlcv(coin_pair, timeframe)
    # update timestamp to human readable timestamp
    data = [[binance.iso8601(candle[0])] + candle[1:] for candle in data]
    header = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
    df = pd.DataFrame(data, columns=header)
    return df


def create_stock(historical_data):
    """Create StockData from historical data 
    """
    stock  = Sdf.retype(historical_data)
    return stock


data = get_historical_data("BTC/USDT", '1h')
stock_data = create_stock(data)

In [33]:
stock_data

,timestamp,open,high,low,close,volume
0,2021-11-22T13:00:00.000Z,57159.42,59444.00,56580.01,58853.92,6278.57949
1,2021-11-22T14:00:00.000Z,58853.92,59023.98,58130.00,58244.41,2517.35914
2,2021-11-22T15:00:00.000Z,58247.28,58457.76,57937.03,58194.79,1818.42684
3,2021-11-22T16:00:00.000Z,58194.78,58250.00,57620.00,57716.56,1961.24426
4,2021-11-22T17:00:00.000Z,57716.56,57776.88,57288.48,57439.62,1520.69259
...,...,...,...,...,...,...
495,2021-12-13T04:00:00.000Z,48873.20,49100.00,48751.96,49025.08,783.08517
496,2021-12-13T05:00:00.000Z,49026.65,49350.90,48851.47,49238.03,1076.26977
497,2021-12-13T06:00:00.000Z,49238.03,49304.00,48872.46,48936.88,905.81189
498,2021-12-13T07:00:00.000Z,48936.88,48995.94,48554.77,48740.01,1284.61901


In [37]:
# 24시간 거래량
last_items = stock_data.tail(24)
print(last_items)
day_volume_self_calculated = last_items['volume'].sum()
print(day_volume_self_calculated)

# better way to do it
ticker = binance.fetch_ticker(coin_pair)
day_volume = ticker["baseVolume"]
print(day_volume)

# quote volume (expressed in BTC value)
day_volume_in_btc = ticker["quoteVolume"]
print(day_volume_in_btc)

                    timestamp      open      high       low     close  \
476  2021-12-12T09:00:00.000Z  48965.01  49150.00  48710.66  48806.40   
477  2021-12-12T10:00:00.000Z  48806.39  48953.26  48638.00  48917.02   
478  2021-12-12T11:00:00.000Z  48917.01  49299.59  48824.00  49124.33   
479  2021-12-12T12:00:00.000Z  49124.33  49632.87  49107.33  49513.38   
480  2021-12-12T13:00:00.000Z  49513.38  49889.00  49491.76  49795.44   
481  2021-12-12T14:00:00.000Z  49795.43  49934.59  49507.09  49862.78   
482  2021-12-12T15:00:00.000Z  49862.79  49910.00  49572.38  49623.26   
483  2021-12-12T16:00:00.000Z  49623.26  49893.88  49501.69  49875.08   
484  2021-12-12T17:00:00.000Z  49871.31  50490.00  49739.47  50251.94   
485  2021-12-12T18:00:00.000Z  50251.94  50777.00  50251.93  50468.71   
486  2021-12-12T19:00:00.000Z  50468.71  50600.00  50014.48  50221.05   
487  2021-12-12T20:00:00.000Z  50221.05  50547.07  50147.80  50270.58   
488  2021-12-12T21:00:00.000Z  50266.13  50317.77  

In [38]:
stock_data['rsi_14']

0            NaN
1       0.000000
2       0.000000
3       0.000000
4       0.000000
         ...    
495    45.028856
496    48.164959
497    44.314673
498    41.953585
499    43.639277
Name: rsi_14, Length: 500, dtype: float64

In [41]:
stock_data['macd']

0        0.000000
1      -13.674904
2      -18.963255
3      -38.170072
4      -58.823272
          ...    
495      1.848596
496    -10.050540
497    -43.282051
498    -84.529621
499   -107.696582
Name: macd, Length: 500, dtype: float64

In [42]:

signal = stock_data['macds'] # Your signal line
macd   = stock_data['macd']  # The MACD that need to cross the signal line
advice = ["No data"]    # Since you need at least two hours in the for loop

for i in range(1, len(signal)):
    # If the MACD crosses the signal line upward
    if macd[i] > signal[i] and macd[i - 1] <= signal[i - 1]:
        advice.append("BUY")
    # The other way around
    elif macd[i] < signal[i] and macd[i - 1] >= signal[i - 1]:
        advice.append("SELL")
    # Do nothing if not crossed
    else:
        advice.append("HOLD")

stock_data['advice'] = advice
print(stock_data['advice'])

0      No data
1         SELL
2         HOLD
3         HOLD
4         HOLD
        ...   
495       HOLD
496       HOLD
497       HOLD
498       HOLD
499       HOLD
Name: advice, Length: 500, dtype: object
